# 高性能的Python

对于**性能**关键应用，总应该检查两件事：
1. 使用正确的范型
2. 使用正确的高性能库

许多**高性能库**可以加速Python代码的执行：
+ Cython， 用于合并Python和C语言静态编译范型
+ IPython.parallel，用于在本地或者集群上并行执行代码/函数
+ numexpr，用于快速计算
+ multiprocessing，Python内建的（本地）并行处理模块
+ Numba，用于为CPU动态编译Python代码
+ NumbaPro，用于为多核CPU和GPU动态编译Python代码

主要内容是比较相同算法的不同实现。定义一个方便的函数，可以系统地比较在相同或者不同数据集上执行不同函数的性能。

In [1]:
from timeit import repeat
def perf_comp_data(func_list, data_list, rep=3, number=1):
    '''Function to compare the comperfance of different functions.
    Args:
    func_list: list
        list with function names as strings
    data_list: list
        list with data set names as strings
    rep: int
        number of repetitions of the while comparsion
    number: int
        number of ececutions for every function'''
    res_list = {}
    for name in enumerate(func_list):
        stmt = name[1] + '(' + data_list[name[0]] + ')'  # function to test
        setup = "from __main__ import " + name[1] + ',' + data_list[name[0]]  # setup code
        results = repeat(stmt=stmt, setup=setup, repeat=rep, number=number)  # a list, element is execution time per repetition
        print results
        res_list[name[1]] = sum(results) / rep  # mean
        # print res_list
    res_sort = sorted(res_list.items(), key=lambda p: p[1])  # sorted by time
    print res_sort
    for item in res_sort:
        rel = item[1] / res_sort[0][1]  # relative
        print 'function: ' + item[0] + ', av. time sec: %9.5f, ' % item[1] + 'relative: %6.1f' % rel

>可以把所有的print都注释掉。

> timeit.repeat(stmt='pass', setup='pass', timer=<default timer>, repeat=3, number=1000000)创建一个Timer实例。

> &emsp;&emsp;stmt: 需要测量的语句或函数（要执行的代码）

> &emsp;&emsp;setup: 初始化代码或构建环境的导入语句（执行代码的准备工作，不计入时间，一般是import之类的）

> &emsp;&emsp;timer: 计时函数，平台相关。Windows: time.clock()  Linux: time.time()

> &emsp;&emsp;repeat: specifies how many times to call tiemit.timeit()，重复执行次数

> &emsp;&emsp;number: specifies the number argument for timeit.timeit(),每一次测量中语句被执行的次数（要执行代码多少遍）

> &emsp;&emsp;Returns: 返回一个列表，表示执行每遍的时间（元素个数=repeat）。可以据此取min、max、mean，因为测得的时间是系统经过的时间，有可能被  正在运行的其他程序影响，所以min可能最有用。本例中取的是mean。

## 1 Python范型与性能

大数据集上的数值计算相当费时。举个例子，我们想要在包含50万个数值的数组上求取某个复杂数学表达式:

$$y = \sqrt{|cos(x)|}+sin(2+3*x)$$

每次计算都会带来一定的计算负担，除此之外没有任何的特殊含义。


Python函数：

In [2]:
from math import *
def f(x):
    return abs(cos(x)) ** .5 + sin(4+3*x)

使用`range`函数高效地生成一个包含50万个数值的列表对象：

In [3]:
a_list = range(500000)

第一个实现，函数`f1()`在整个数据集上循环，在结果列表对象中附加单独函数求职结果：**（1 包含显式循环的标准Python函数）**

In [4]:
def f1(a):  # loop in the data set
    res = []  
    for x in a:
        res.append(f(x))
    return res

第二种实现，使用迭代：**（2 包含隐含循环的迭代子方法）**

In [5]:
def f2(a):  # iteration
    return [f(x) for x in a]

第三种实现，使用`eval`函数：**（3 包含隐含循环、使用eval的迭代子方法）**

In [6]:
def f3(a):  # function: eval
    ex = 'abs(cos(x)) ** .5 + sin(4+3*x)'
    return [eval(ex) for x in a]

第四种实现，使用Numpy的向量化技术。数据是一个ndarry对象而不是list。没有循环，所以有发生在Numpy级别而不是Python级别：**（4 Numpy向量化技术）**

In [7]:
import numpy as np
a_np = np.array(a_list)
def f4(a):
    return (np.abs(np.cos(a))) ** .5 + np.sin(4+3*a) # element-wise

第五种实现，是使用专门的**numexpr库**来求数值表达式的值，这个库内建了**多线程执行支持**：**（5 numexpr单线程实现）**

In [8]:
import numexpr as ne
def f5(a):
    ex =  'abs(cos(a)) ** .5 + sin(4+3*a)'
    ne.set_num_threads(1)  # single-thread
    return ne.evaluate(ex)

> ne.set_num_threads(nthreads) ->Set a number of threads to be used in operations. Returns the previous setting for the number of threads.
>ne.evaluate(\*, \**) ->Evaluate a simple array expression element-wise, using the new iterator.

**（6 numexpr多线程实现）**

In [9]:
def f6(a):  # multi-thread
    ex =  'abs(cos(a)) ** .5 + sin(4+3*a)'
    ne.set_num_threads(16)
    return ne.evaluate(ex)

使用`%%time`(给出cell的代码运行一次所花费的时间)计算总执行时间

In [10]:
%%time
r1 = f1(a_list)
r2 = f2(a_list)
r3 = f3(a_list)
r4 = f4(a_np)
r5 = f5(a_np)
r6 = f6(a_np)

Wall time: 7.46 s


In [11]:
np.allclose(r1, r3)

True

In [12]:
np.allclose(r2, r5)

True

> 可以用Numpy函数allclose可以轻松地检查两个（类）ndarray对象是否包含相同的数据。

不同实现执行速度的对比：

In [13]:
func_list = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6']
data_list = ['a_list', 'a_list', 'a_list', 'a_np', 'a_np', 'a_np']
perf_comp_data(func_list=func_list, data_list=data_list)

[0.30793639506173065, 0.2870289382716038, 0.28481422222222363]
[0.2719648395061718, 0.2627022222222237, 0.2629760000000019]
[6.856801580246916, 6.7821368888888856, 6.793328592592594]
[0.03694103703703888, 0.03617382716049633, 0.03631209876543551]
[0.01396424691358078, 0.009987950617286856, 0.009987555555554195]
[0.007250567901230909, 0.008183308641974918, 0.006720395061726947]
[('f6', 0.007384757201644258), ('f5', 0.011313251028807278), ('f4', 0.03647565432099024), ('f2', 0.26588102057613244), ('f1', 0.2932598518518527), ('f3', 6.810755687242799)]
function: f6, av. time sec:   0.00738, relative:    1.0
function: f5, av. time sec:   0.01131, relative:    1.5
function: f4, av. time sec:   0.03648, relative:    4.9
function: f2, av. time sec:   0.26588, relative:   36.0
function: f1, av. time sec:   0.29326, relative:   39.7
function: f3, av. time sec:   6.81076, relative:  922.3


**Conclusion**

胜者：多线程`numexpr`实现`f6`。当然，它的速度优势取决于可用核心数量。向量化`numpy`版本`f4`慢于`f5`。纯`Python`实现`f1`和`f2`比胜者慢几十倍。`f3`是最慢的版本，因为对这样大量的求值运算使用`eval`会造成巨大的负担。在`numexpr`的例子中，基于字符串的表达式计算运行一次之后被编译供以后使用；而使用`Python eval`函数，这样一操作要进行50万次。**`numexpr`比`numpy`要快快。在执行大量预算和对大数组进行操作时一定要使用`numexpr`。**

## 2 内存布局与性能

numpy可以为每个ndarray对象指定所谓的dtype，如np.int32或者f8。numpy还允许在初始化ndarray对象时从两种不同的**内存布局（C或Fortran）**中选择。
根据对象结构，某种布局可能比另一种布局更有优势。

In [14]:
import numpy as np
np.zeros((3, 3), dtype=np.float64, order='C')

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

> 初始化numpy ndarray对象的方式可能（根据数组的大小）对这些数组的操作性能产生显著的影响。

**ndarray对象的初始化(通过`np.zeros`或`np.array`)输入参数**

+ *shape*: &emsp;&emsp;整数(int)。整数序列或者引用另一个numpy.ndarray
+ *dtype(optional)*:&emsp;numpy.dtype--用于numpy.ndarray对象的numpy特定数据类型
+ *order(optional)*:&emsp;元素在内存中存储的顺序：C表示类似C(行优先)(default)，F表示类似Fortran(列优先)

In [15]:
c = np.array([[1,1, 1], [2, 2, 2], [3, 3, 3]], order='C')

> 本例中，1， 2， 3相邻存储。

In [16]:
f = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]], order='F')

> 本例中，数据存储方式是1， 2， 3在每列中相邻存储。

**看看内存布局在数组很大时可能带来什么差异：**

In [17]:
x = np.random.standard_normal((3, 1500000))
c = np.array(x, order='C')
f = np.array(x, order='F')
x = 0.0

类似C的布局

In [18]:
# sum(axis=0)
%timeit c.sum(axis=0)

100 loops, best of 3: 10.4 ms per loop


In [19]:
# sum(axis=1)
%timeit c.sum(axis=1)

100 loops, best of 3: 5.63 ms per loop


> 在第一个轴上计算比在第二个轴上慢了近一倍。

In [20]:
# standard deviation(axis=0)
%timeit c.std(axis=0)

10 loops, best of 3: 48.1 ms per loop


In [21]:
# standard deviation(axis=0)
%timeit c.std(axis=1)

10 loops, best of 3: 30.4 ms per loop


类似Fortran的布局

In [22]:
# sum(axis=0)
%timeit f.sum(axis=0)

10 loops, best of 3: 19.5 ms per loop


In [23]:
# sum(axis=1)
%timeit f.sum(axis=1)

10 loops, best of 3: 22.9 ms per loop


> 尽管和其他布局相比绝对更慢，但是两个轴的相对差值并不太大

In [24]:
# standard deviation(axis=0)
%timeit f.std(axis=0)

10 loops, best of 3: 83.1 ms per loop


In [25]:
# standard deviation(axis=1)
%timeit f.std(axis=1)

10 loops, best of 3: 73.1 ms per loop


> 同样与类似C的布局相比，这种布局的性能更差。两个轴之间的差距很小，但是不像其他布局那样显著。

**Conclusion**

一般来说，**类似C的布局表现更好**——这也是numpy ndarray对象没有指定的时候默认使用这种内存布局额原因。

## 3 并行计算

pass

## 4 多处理

**在本地并行执行代码**，这是**`Python multiprocessing`模块**用武之地

**几何布朗运动(Geometric Brownian Motion, GBM)**
> 在连续时间情况下的随机过程，其中随机变量的对数遵循布朗运动。在金融中，用来在布莱克·舒尔斯定价模型中模仿股票价格。
> A随机过程$S_t$在满足以下随机微分方程(SDE)的情况下被认为遵循几何布朗运动：

> $$dS_t = \mu S_td_t + \sigma S_tdW_t$$

>&emsp;&emsp;&emsp;&emsp;$W_t$：维纳过程或者说是布朗运动&emsp;&emsp;$\mu$：漂移百分比(常量)&emsp;&emsp; $\sigma$：波动百分比(常量)

> 给定初始值$S_0$，根据伊藤积分，有如下解：
> $$S_t = S_0{\rm e}^{(\mu - 0.5\sigma^2)t+\sigma W_t}$$

**布朗运动(Brownian Motion)**
> 微小粒子表现出的无规则运动。由于颗粒受到液体分子碰撞的不平衡力作用而引起的。

> 大物体(如线度0.01mm)将从各方面受到运动原子的冲击，打击非常频繁，概率定律使之相互补偿，故他们不移动。微小粒子受到的打击太少，以至于无法补偿。

> 布朗运动是液体分子处于不停顿无规则热运动的宏观表现。

模仿几何布朗运动

In [26]:
import math
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
def simulateGeometricBrownianMotion(p):
    M, I = p  # time steps, paths
    S0 = 100; r = .05; sigma = .2; T = 1.0   # model parameters
    dt =  T / M
    paths = np.zeros((M+1, I))
    paths[0] = S0
    for t in range(1, M+1):
        paths[t] = paths[t-1] * np.exp((r-.5*sigma**2)*dt + sigma*math.sqrt(dt)*np.random.standard_normal(I))
    return paths

In [28]:
# 该函数返回以M和I为参数的模拟路径
paths = simulateGeometricBrownianMotion((5, 2))
paths

array([[ 100.        ,  100.        ],
       [ 110.72014166,   95.76332374],
       [  93.76504308,   94.13789956],
       [ 105.82017234,   94.85533654],
       [ 115.02735797,  102.96876957],
       [ 151.73983758,  109.67530707]])

在一个具有8个核心的电脑上根据参数值实现一个测试，打算进行100次模拟：

In [29]:
I = 10000  # number of paths
M = 100    # number of time steps
t = 100    # number of tasks/simulates

In [30]:
"""from time import time
times = []
for w in range(1, 8):
    t0 = time()
    pool = mp.Pool(processes=w)   # the pool of works
    result = pool.map(simulateGeometricBrownianMotion, t*[(M, I), ])  # the mapping of the function to the list of parameter tuples
    times.append(time() - t0)
"""

'from time import time\ntimes = []\nfor w in range(1, 8):\n    t0 = time()\n    pool = mp.Pool(processes=w)   # the pool of works\n    result = pool.map(simulateGeometricBrownianMotion, t*[(M, I), ])  # the mapping of the function to the list of parameter tuples\n    times.append(time() - t0)\n'

## 5 动态编译

**Numba**是开源的、Numpy感知的优化Python代码编译器。使用LLVM编译器基础架构，将Python字节代码编译专门用于Numpy运行时和Scipy模块的机器代码。
> LLVM(原Low Level Virtual Machine，低级虚拟机)的缩写，现在“它使用该项目的全名”。

影响性能的问题：包含嵌套循环的算法

In [31]:
import math
def f_py(I, J):
    res = 0
    for i in range(I):
        for j in range(J):
            res += int(math.cos(math.log(1)))
    return res

In [32]:
I, J = 5000, 5000
%time f_py(I, J)

Wall time: 8.9 s


25000000

In [33]:
import numpy as np
def f_np(I, J):
    a = np.ones((I, J), dtype=np.float64)
    return int(np.sum(np.cos(np.log(a)))), a

In [34]:
%time res, a = f_np(I, J)

Wall time: 478 ms


> 这种方法快的多，但是并不能真正高效地利用内存。

In [35]:
a.nbytes

200000000

> ndarray会消耗200MB内存

考虑到RAM的数量，很容易选择I和J使Numpy方法变得不可行。Numba提供了一种有吸引力的替代方法，可以解决这种循环结构的性能问题，同时保持纯Python方法的内存效率。**使用Numba，只需要对Python函数应用`jit`函数，生成该函数的Python可调用编译版本**。

In [43]:
import numba as nb
f_nb = nb.jit(f_py)

In [44]:
%time res = f_nb(I, J)

Wall time: 54 ms


> 这个新函数可以直接从Python解释程序中调用，实现比Numpy向量化更显著的加速效果。

In [45]:
func_list = ['f_py', 'f_np', 'f_nb']
data_list = 3 * ['I, J']
perf_comp_data(func_list, data_list)

[8.821778172839515, 8.773172543209853, 8.784802765432119]
[0.48458587654317853, 0.47801797530865997, 0.4796871111111045]
[3.555555565526447e-06, 7.901234084783937e-07, 3.950617610826157e-07]
[('f_nb', 1.5802469116958189e-06), ('f_np', 0.480763654320981), ('f_py', 8.79325116049383)]
function: f_nb, av. time sec:   0.00000, relative:    1.0
function: f_np, av. time sec:   0.48076, relative: 304233.3
function: f_py, av. time sec:   8.79325, relative: 5564479.3


**Conclusion**

嵌套循环实现的Numba版本是目前最快的方案，甚至远快于Numpy的向量化版本。Python版本比其他两种版本慢得多。

> 改善（数值算法）性能的许多方法都需要花费可观的精力。利用Python和Numba，就有了需要最少精力的一种方法——一般来说，只需要导入该库和一行附加代码。他不能用于所有类型算法，但是往往值得（简单地）一试，有时候确实能够快速取得效果。

## 6 用Cython进行静态编译

**Numba的优势是对任意函数应用该方法好不费力**。但是，Numba只能为*某些问题*“毫不费力”地产生显著的性能改善。另一种方法更为灵活，但是也需要更多的精力，这就是通过**Cython的静态编译**。实际上，Cython是Python和C语言的混血儿。从Python的角度来看，需要注意的主要不同是静态类型声明（和C语言相同）和一个单独的编译步骤（和任何编译语言相同）。

实例：嵌套循环再次简单地返回循环次数，和前面相比，这次内循环次数由外循环次数放大。

In [46]:
def f_py(I, J):
    res = 0.0  # we work on a float object
    for i in range(I):
        for j in range(J*I):
            res += 1
    return res

In [48]:
I, J = 500, 500  # if use Numpy shape is (500, 250000)
%time f_py(I, J)

Wall time: 6.24 s


125000000.0

采用相同的函数并引入用于Cython的静态类型声明。这个Cython文件的扩展名为.pyx。

```python
%load_ext cythonmagic
%%cython-pyximport nested_loop
def f_cy(int I, int J):
    cdef double res = 0  # double float much slower than int or float
    for i in range(I):
        for j in range(J * I):
            res += 1
    return res
    
%time res = f_cy(I, J)
```

> 代码有问题，找了很长时间没找到解决办法。

## 7 GPU
pass

## 8 结语

Python生态系统提供了一些改进代码性能的手段
1. **范型**&emsp;&emsp;&emsp;在给定问题上，某些Python范型比其他范型性能更好
2. **库**&emsp;&emsp;&emsp;&emsp;有由于不同类型问题的大量库，这些库（如numexpr）往往导致适合于该库范围的问题得到性能更好额解决方案
3. **编译**&emsp;&emsp;&emsp;有一些强大的编译解决方案，包括静态（如Cython）和动态（如Numba）方案
4. **并行化**&emsp;&emsp;Python有内建的并行化功能（如numexpr），而其他一些库允许我们利用多核CPU、整个集群（IPython.parallel）或者GPU（Numbapro）的能力。
    

Python生态系统的重大优势之一是所有方法通常都很容易实现，需要额外的精力通常很少。性能的改进是唾手可得的